# 1. Introdução e Objetivo

Neste notebook, exploramos o algoritmo **DBSCAN** (Density-Based Spatial Clustering of Applications with Noise) para a detecção de anomalias no dataset de cartões de crédito.

Como o DBSCAN é um algoritmo de aprendizado não supervisionado baseado em densidade, ele é ideal para identificar "ruídos" (fraudes) que não pertencem a nenhum cluster denso de transações normais.

**Objetivos:**
1. Validar o uso de **PCA** para redução de dimensionalidade e otimização de performance.
2. Determinar os hiperparâmetros ideais (`eps` e `min_samples`) utilizando o método do cotovelo (K-Distance Graph).
3. Validar o modelo em uma amostra de teste sincronizada com a equipe.

In [ ]:
import os
import pandas as pd

# Inicializa para evitar erro lá na frente
df = None

# Tenta ler o dado processado (Ideal para o GitHub)
path_processed = '../data/processed/X_test_processed.csv'
path_raw_local = '../data/raw/creditcard.csv'
path_colab = '/content/drive/MyDrive/AMCD/creditcard.csv'

if os.path.exists(path_processed):
    print("Modo Integração: Lendo dados processados...")
    X = pd.read_csv(path_processed)
    # Se ler o processado, NÃO precisa normalizar de novo, apenas PCA
    NEED_SCALING = False 
elif os.path.exists(path_raw_local):
    print("Modo Dev Local: Lendo dados brutos...")
    df = pd.read_csv(path_raw_local)
    X = df.drop(columns=['Class', 'Time'])
    NEED_SCALING = True
else:
    print("Modo Colab: Lendo do Drive...")
    df = pd.read_csv(path_colab)
    X = df.drop(columns=['Class', 'Time'])
    NEED_SCALING = True

# 2. Análise de Distância (O Gráfico do Cotovelo)

Para definir o raio de vizinhança (`eps`), utilizamos o método do **Gráfico de Distância-K**. Calculamos a distância de cada ponto para o seu $k$-ésimo vizinho mais próximo.

* **Min_samples (k):** Definido como 14.
* **PCA:** Aplicamos PCA reduzindo para 10 componentes para viabilizar o cálculo de distâncias euclidianas.

O ponto de "cotovelo" (curvatura máxima) no gráfico indica o valor ideal de Epsilon, onde os pontos deixam de ser "vizinhos" e começam a ser considerados outliers.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import DBSCAN
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# Só aplica scaler se leu o dado bruto (se leu processado, já vem normalizado)
if NEED_SCALING:
    scaler = StandardScaler()
    X_input = scaler.fit_transform(X)
else:
    X_input = X

# PCA é sempre sua responsabilidade
pca = PCA(n_components=10)
X_pca = pca.fit_transform(X_input)

In [ ]:
# --- CÁLCULO DO K-DISTANCE GRAPH (Método do Cotovelo) ---

# Configuração
k = 14  # Min_samples definido

# Amostragem segura:
# Se o dataset for muito grande (>50k linhas), pegamos uma amostra de 20% para o gráfico não travar.
# Se for o dataset de teste processado (aprox 42k), usamos tudo.
if len(X_pca) > 50000:
    print("Dataset grande detectado. Usando amostra de 20% para visualização...")
    np.random.seed(42)
    sample_indices = np.random.choice(len(X_pca), int(len(X_pca)*0.2), replace=False)
    X_plot = X_pca[sample_indices]
else:
    print("Dataset de tamanho comportado. Usando todos os pontos para visualização.")
    X_plot = X_pca

# Cálculo dos Vizinhos
print(f"Calculando distâncias para k={k}...")
nbrs = NearestNeighbors(n_neighbors=k).fit(X_plot)
distances, indices = nbrs.kneighbors(X_plot)

# Ordenação para o gráfico
distance_desc = sorted(distances[:, k-1])

# Função auxiliar para encontrar o ponto de curvatura máxima (Knee/Elbow Point)
def get_knee_point(values):
    n_points = len(values)
    all_coords = np.vstack((range(n_points), values)).T
    first_point = all_coords[0]
    line_vec = all_coords[-1] - all_coords[0]
    line_vec_norm = line_vec / np.sqrt(np.sum(line_vec**2))
    vec_from_first = all_coords - first_point
    scalar_product = np.sum(vec_from_first * np.tile(line_vec_norm, (n_points, 1)), axis=1)
    vec_from_first_parallel = np.outer(scalar_product, line_vec_norm)
    vec_to_line = vec_from_first - vec_from_first_parallel
    dist_to_line = np.sqrt(np.sum(vec_to_line ** 2, axis=1))
    idx_of_best_point = np.argmax(dist_to_line)
    return values[idx_of_best_point]

# Encontrar valor sugerido
eps_sugerido = get_knee_point(distance_desc)
print(f"--- RESULTADO ---")
print(f"Epsilon sugerido pelo método matemático: {eps_sugerido:.4f}")

# Plotagem
plt.figure(figsize=(10, 6))
plt.plot(distance_desc)
plt.axhline(y=eps_sugerido, color='r', linestyle='--', label=f'Eps Sugerido ({eps_sugerido:.2f})')
plt.title('Gráfico de Distância-K (Método do Cotovelo)')
plt.ylabel('Distância Epsilon')
plt.xlabel('Pontos (ordenados por distância)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# 3. Validação do Modelo

Agora aplicamos o DBSCAN com o valor de Epsilon escolhido (`eps=2.96`) para verificar a performance.

* **Se estivermos no Colab (Modo Raw):** Faremos a divisão "Treino/Teste" simulada para calcular Recall e Precisão, pois temos as labels originais.
* **Se estivermos no GitHub (Modo Integração):** O código apenas rodará a predição no conjunto `X_test_processed` (sem labels), simulando o ambiente de produção.

In [ ]:
# --- EXECUÇÃO E VALIDAÇÃO ---

# Parâmetros Finais
EPS_FINAL = 2.9619
MIN_SAMPLES = 14

print(f"Rodando DBSCAN (eps={EPS_FINAL}, min_samples={MIN_SAMPLES})...")
print("Isso pode levar alguns segundos...")

# Se estivermos rodando no COLAB (Dataset Bruto com Labels 'Class')
# Precisamos criar o conjunto de teste igual ao do grupo para validar
if df is not None and 'Class' in df.columns:
    print("\n[MODO VALIDAÇÃO] Dataset completo com labels encontrado.")
    print("Recriando o split de teste oficial (42.722 amostras) para calcular métricas...")
    
    # Recria o split (Logica do Integrante 1)
    y_real = df['Class']
    # 1. Tira Treino (30%)
    _, X_temp_pca, _, _, _, y_temp = train_test_split(
        X_pca, df.index, y_real, test_size=0.30, stratify=y_real, random_state=42
    )
    # 2. Tira Validação (50% do resto) -> Sobra Teste (50%)
    _, X_teste_final, _, _, _, y_teste_final = train_test_split(
        X_temp_pca, range(len(X_temp_pca)), y_temp, test_size=0.50, stratify=y_temp, random_state=42
    )
    
    # Roda DBSCAN na fatia de teste
    db = DBSCAN(eps=EPS_FINAL, min_samples=MIN_SAMPLES, n_jobs=-1)
    labels = db.fit_predict(X_teste_final)
    
    # Métricas
    y_pred = [1 if x == -1 else 0 for x in labels]
    
    print("\n--- RELATÓRIO DE CLASSIFICAÇÃO (Amostra de Teste) ---")
    print(classification_report(y_teste_final, y_pred, target_names=['Normal', 'Fraude']))
    
    cm = confusion_matrix(y_teste_final, y_pred)
    tn, fp, fn, tp = cm.ravel()
    print(f"Matriz de Confusão:")
    print(f"Fraudes Detectadas (Recall): {tp}")
    print(f"Falsos Positivos: {fp}")
    print(f"Fraudes Perdidas: {fn}")

else:
    # MODO INTEGRAÇÃO (Apenas X disponível)
    print("\n[MODO PRODUÇÃO] Apenas features disponíveis (sem labels).")
    print(f"Rodando predição em {len(X_pca)} amostras...")
    
    db = DBSCAN(eps=EPS_FINAL, min_samples=MIN_SAMPLES, n_jobs=-1)
    labels = db.fit_predict(X_pca)
    
    n_anomalies = np.sum(labels == -1)
    print(f"Concluído! Anomalias detectadas: {n_anomalies}")
    print("Para ver métricas de acerto, rode no Colab com o dataset original.")

# 4. Conclusão e Definição de Parâmetros

Com base na experimentação acima, definimos a configuração final para o script de produção:

1.  **Pré-processamento:** Normalização (`StandardScaler`) seguida de **PCA (10 componentes)**. A redução de dimensionalidade provou-se essencial para viabilizar o cálculo da matriz de distâncias.
2.  **Hiperparâmetros:**
    * `eps = 2.9619`: Identificado visual e matematicamente pelo método do cotovelo.
    * `min_samples = 14`: Valor robusto para evitar que micro-clusters de ruído sejam considerados normais.
3.  **Resultados:** O modelo demonstrou capacidade de detectar fraudes (Recall) no conjunto de teste, ainda que com uma taxa esperada de falsos positivos, característica intrínseca de métodos de densidade em dados desbalanceados.

**Próximos Passos:**
O algoritmo validado aqui foi implementado no script `src/models/dbscan.py`, configurado para processar o conjunto de teste oficial e gerar o arquivo de submissão `outputs/dbscan_predictions.csv`.